In [1]:
# pip install openpyxl
# pip install pandas

In [2]:
import pandas as pd
import configparser
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

# Setting up connection to Snowflake via personal work account
config = configparser.ConfigParser()
config.read('toad.cfg')
snowflake_pass = config['SNOWFLAKE']['secret_passphrase']

with open("/tech/appl/default/user/pa08042e/rsa_snow.p8", "rb") as key:
    private_key= serialization.load_pem_private_key(
        key.read(),
        password= snowflake_pass.encode(),
        backend=default_backend()
    )

pkb = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption())

ctx = snowflake.connector.connect(
    user='pa08042',
    account='hfsg_prod.us-east-1',
    private_key=pkb,
    database='user_db',
    schema='AD1_PA08042'
    )

In [3]:
# Creating an empty dataframe with correct schema to append Snowflake state data to
full_data = pd.DataFrame(columns = ['POL_ID', 'RISK_ST_ABBR', 'ORIG_POL_EFF_DT', 'POL_EFF_DT', 'TRANS_STAT_DESC', 'CLEAN_DIRTY_CD',
                                    'POL_TTL_SPEED_MNR_VIO_CNT', 'POL_TTL_NSPD_MNR_VIO_CNT', 'UW_ELIG_POL_AFA_CNT', 'UW_ELIG_POL_NAF_ACCID_CNT',
                                    'UW_ELIG_POL_COMP_CLM_CNT', 'UW_ELIG_POL_MLTRY_PRMT_CNT', 'UW_ELIG_POL_PERM_USE_ACCID_CNT', 'UW_ELIG_RENTL_TOW_GLASS_CNT',
                                    'DERIVED_TTL_ACCID_VIO_COMP_CNT', 'DERIVED_YES_NO_FLAG'])

# This dictionary stores the renewal dates associated with each state that has the UW Clean/Dirty functionality
renewal_state_dict = {'\'IN\'': '\'2024-09-09\'', 
                      '\'TN\',\'MO\'': '\'2024-09-01\'', 
                      '\'AZ\'': '\'2024-09-14\'',
                     '\'CO\', \'IA\', \'ID\', \'LA\', \'OK\', \'UT\', \'WY\'': '\'2024-11-24\'',
                     '\'NE\', \'MS\'': '\'2024-12-02\'', 
                      '\'MD\', \'VT\', \'WV\'': '\'2024-12-07\'', 
                      '\'KY\'': '\'2024-12-27\'',
                     '\'MN\', \'SD\'': '\'2024-12-22\'',
                     '\'WI\'': '\'2024-12-23\''}

# Iterates through renewal dictionary
for key, value in renewal_state_dict.items():
    
    # Takes every item in dictionary and uses the key/value as filters for the query that is based on ThoughtSpot answer
    query = f"""
    SELECT POL_ID, RISK_ST_ABBR, 
    TO_CHAR(TO_DATE(ORIG_POL_EFF_DT), 'Mon YYYY') AS ORIG_POL_EFF_DT, 
    TO_CHAR(TO_DATE(POL_EFF_DT), 'Mon YYYY') AS POL_EFF_DT,
    TRANS_STAT_DESC, CLEAN_DIRTY_CD,
    POL_TTL_SPEED_MNR_VIO_CNT, POL_TTL_NSPD_MNR_VIO_CNT,
    ZEROIFNULL(UW_ELIG_POL_AFA_CNT) AS UW_ELIG_POL_AFA_CNT,
    ZEROIFNULL(UW_ELIG_POL_NAF_ACCID_CNT) AS UW_ELIG_POL_NAF_ACCID_CNT,
    ZEROIFNULL(UW_ELIG_POL_COMP_CLM_CNT) AS UW_ELIG_POL_COMP_CLM_CNT,
    ZEROIFNULL(UW_ELIG_POL_MLTRY_PRMT_CNT) AS UW_ELIG_POL_MLTRY_PRMT_CNT,
    ZEROIFNULL(UW_ELIG_POL_PERM_USE_ACCID_CNT) AS UW_ELIG_POL_PERM_USE_ACCID_CNT,
    ZEROIFNULL(UW_ELIG_RENTL_TOW_GLASS_CNT) AS UW_ELIG_RENTL_TOW_GLASS_CNT,
    ZEROIFNULL(UW_ELIG_POL_AFA_CNT + UW_ELIG_POL_NAF_ACCID_CNT + 
    UW_ELIG_POL_COMP_CLM_CNT + UW_ELIG_POL_MLTRY_PRMT_CNT + UW_ELIG_POL_PERM_USE_ACCID_CNT) AS DERIVED_TTL_ACCID_VIO_COMP_CNT,
    CASE WHEN DERIVED_TTL_ACCID_VIO_COMP_CNT = 0 THEN 'No Acc/Viol/Comp' ELSE 'Yes Acc/Viol/Comp' END AS DERIVED_YES_NO_FLAG
    FROM "PRD_PL_DB"."APP_DCPA_DM"."AUTO_POLICY_LATST_VW"
    WHERE POL_EFF_DT >= {value} 
    AND RISK_ST_ABBR IN ({key})
    AND ORIG_POL_EFF_DT < {value}
    """
    
    # Gets policies from Snowflake
    cs = ctx.cursor()
    try:
        cs.execute(query)
        df_temp = cs.fetch_pandas_all()
    finally:
        cs.close()

    # Appends the records pulled from Snowflake 
    full_data = pd.concat([full_data, df_temp], ignore_index = True)

In [4]:
full_data.shape

(35455, 16)

In [5]:
# Writes the policy data to Snowflake
full_data.reset_index(drop=True, inplace=True)
success, nchunks, nrows, _ = write_pandas(ctx, full_data,'FULL_UW_PYTHON', table_type = 'transient', use_logical_type=True, overwrite=True)
print(success)

True


In [6]:
# Creates the dashboard's table starting with July states (AZ, IN, MO, TN)
july_query = """
CREATE OR REPLACE TRANSIENT TABLE USER_DB.AD1_PA08042.DATA_JULY
AS
(
SELECT 
UW_AUTO.POL_ID, 
EARNED_MONTH,
MIN(UW_AUTO.RISK_ST_ABBR) AS RISK_ST_ABBR,
MIN(UW_AUTO.ORIG_POL_EFF_DT) AS ORIG_POL_EFF_DT,
MIN(UW_AUTO.POL_EFF_DT) AS LATEST_POL_EFF_DT,
MIN(UW_AUTO.TRANS_STAT_DESC) AS TRANS_STAT_DESC,
MIN(UW_AUTO.CLEAN_DIRTY_CD) AS CLEAN_DIRTY_CD,
MIN(UW_AUTO.POL_TTL_SPEED_MNR_VIO_CNT) AS POL_TTL_SPEED_MNR_VIO_CNT,
MIN(UW_AUTO.POL_TTL_NSPD_MNR_VIO_CNT) AS POL_TTL_NSPD_MNR_VIO_CNT,
MIN(UW_AUTO.UW_ELIG_POL_AFA_CNT) AS UW_ELIG_POL_AFA_CNT,
MIN(UW_AUTO.UW_ELIG_POL_NAF_ACCID_CNT) AS UW_ELIG_POL_NAF_ACCID_CNT,
MIN(UW_AUTO.UW_ELIG_POL_COMP_CLM_CNT) AS UW_ELIG_POL_COMP_CLM_CNT,
MIN(UW_AUTO.UW_ELIG_POL_MLTRY_PRMT_CNT) AS UW_ELIG_POL_MLTY_PRMT_CNT,
MIN(UW_AUTO.UW_ELIG_POL_PERM_USE_ACCID_CNT) AS UW_ELIG_POL_PERM_USE_ACCID_CNT,
MIN(UW_AUTO.UW_ELIG_RENTL_TOW_GLASS_CNT) AS UW_ELIG_RENTL_TOW_GLASS_CNT,
MIN(UW_AUTO.DERIVED_TTL_ACCID_VIO_COMP_CNT) AS DERIVED_TTL_ACCID_VIO_COMP_CNT,
MIN(UW_AUTO.DERIVED_YES_NO_FLAG) AS DERIVED_YES_NO_FLAG,
SUM(AUTO_OP_LOSS.INCUR_CLM_CNT_XCAT) AS INCUR_CLM_CNT_XCAT,
SUM(AUTO_OP_LOSS.DERIV_INCUR_LOSS_AMT_XCAT) AS DERIV_INCUR_LOSS_AMT_XCAT,
SUM(AUTO_OP_LOSS.GLASS_TOW_INCUR_CLM_CNT) AS GLASS_TOW_INCUR_CLM_CNT,
SUM(AUTO_OP_LOSS.GLASS_TOW_INCUR_LOSS_AMT) AS GLASS_TOW_INCUR_LOSS_AMT,
SUM(AUTO_OP_LOSS.DERIV_ADJ_EP_AMT) AS EP,                 
SUM(AUTO_OP_LOSS.RERATED_EP_EPAPR_AMT) AS EPAPR,            
DAY(LAST_DAY(TO_DATE(EARNED_MONTH || '01', 'YYYYMMDD'))) / 
IFF(MOD(SUBSTR(EARNED_MONTH, 1, 4), 4) = 0 AND (MOD(SUBSTR(EARNED_MONTH, 1, 4), 100) != 0 OR MOD(SUBSTR(EARNED_MONTH, 1, 4), 400) = 0), 366, 365)
AS EE                                                    
FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_OPERATIONAL_LOSS" AS AUTO_OP_LOSS
JOIN USER_DB.AD1_PA08042.FULL_UW_PYTHON AS UW_AUTO
ON CAST(AUTO_OP_LOSS.POL_ID AS VARCHAR) = CAST(UW_AUTO.POL_ID AS VARCHAR)
WHERE COV_TYPE_ABBR != 'PUP'
AND RISK_ST_ABBR IN ('AZ', 'IN', 'MO', 'TN')
AND EARNED_MONTH >= '202407' 
GROUP BY UW_AUTO.POL_ID, EARNED_MONTH 
ORDER BY UW_AUTO.POL_ID, EARNED_MONTH   
)
"""

# Adds October states to the dashboard's table (CO, MS, WI, UT, KY, MD, IA, LA, MN, NE, ID, WV, VT, WY, OK, SD)
october_query = """
INSERT INTO USER_DB.AD1_PA08042.DATA_JULY
SELECT 
UW_AUTO.POL_ID, 
EARNED_MONTH,   

MIN(UW_AUTO.RISK_ST_ABBR) AS RISK_ST_ABBR,
MIN(UW_AUTO.ORIG_POL_EFF_DT) AS ORIG_POL_EFF_DT,
MIN(UW_AUTO.POL_EFF_DT) AS LATEST_POL_EFF_DT,
MIN(UW_AUTO.TRANS_STAT_DESC) AS TRANS_STAT_DESC,
MIN(UW_AUTO.CLEAN_DIRTY_CD) AS CLEAN_DIRTY_CD,
MIN(UW_AUTO.POL_TTL_SPEED_MNR_VIO_CNT) AS POL_TTL_SPEED_MNR_VIO_CNT,
MIN(UW_AUTO.POL_TTL_NSPD_MNR_VIO_CNT) AS POL_TTL_NSPD_MNR_VIO_CNT,
MIN(UW_AUTO.UW_ELIG_POL_AFA_CNT) AS UW_ELIG_POL_AFA_CNT,
MIN(UW_AUTO.UW_ELIG_POL_NAF_ACCID_CNT) AS UW_ELIG_POL_NAF_ACCID_CNT,
MIN(UW_AUTO.UW_ELIG_POL_COMP_CLM_CNT) AS UW_ELIG_POL_COMP_CLM_CNT,
MIN(UW_AUTO.UW_ELIG_POL_MLTRY_PRMT_CNT) AS UW_ELIG_POL_MLTY_PRMT_CNT,
MIN(UW_AUTO.UW_ELIG_POL_PERM_USE_ACCID_CNT) AS UW_ELIG_POL_PERM_USE_ACCID_CNT,
MIN(UW_AUTO.UW_ELIG_RENTL_TOW_GLASS_CNT) AS UW_ELIG_RENTL_TOW_GLASS_CNT,
MIN(UW_AUTO.DERIVED_TTL_ACCID_VIO_COMP_CNT) AS DERIVED_TTL_ACCID_VIO_COMP_CNT,
MIN(UW_AUTO.DERIVED_YES_NO_FLAG) AS DERIVED_YES_NO_FLAG,

SUM(AUTO_OP_LOSS.INCUR_CLM_CNT_XCAT) AS INCUR_CLM_CNT_XCAT,
SUM(AUTO_OP_LOSS.DERIV_INCUR_LOSS_AMT_XCAT) AS DERIV_INCUR_LOSS_AMT_XCAT,
SUM(AUTO_OP_LOSS.GLASS_TOW_INCUR_CLM_CNT) AS GLASS_TOW_INCUR_CLM_CNT,
SUM(AUTO_OP_LOSS.GLASS_TOW_INCUR_LOSS_AMT) AS GLASS_TOW_INCUR_LOSS_AMT,

SUM(AUTO_OP_LOSS.DERIV_ADJ_EP_AMT) AS EP, 
SUM(AUTO_OP_LOSS.RERATED_EP_EPAPR_AMT) AS EPAPR,
DAY(LAST_DAY(TO_DATE(EARNED_MONTH || '01', 'YYYYMMDD'))) / 
IFF(MOD(SUBSTR(EARNED_MONTH, 1, 4), 4) = 0 AND (MOD(SUBSTR(EARNED_MONTH, 1, 4), 100) != 0 OR MOD(SUBSTR(EARNED_MONTH, 1, 4), 400) = 0), 366, 365)
AS EE                                              
FROM "DSC_PLBI_DB"."APP_AUTO_PRD"."AUTO_OPERATIONAL_LOSS" AS AUTO_OP_LOSS
JOIN USER_DB.AD1_PA08042.FULL_UW_PYTHON AS UW_AUTO
ON CAST(AUTO_OP_LOSS.POL_ID AS VARCHAR) = CAST(UW_AUTO.POL_ID AS VARCHAR)
WHERE COV_TYPE_ABBR != 'PUP'
AND RISK_ST_ABBR IN 
('CO', 'MS', 'WI', 'UT', 'KY', 'MD', 'IA', 'LA', 'MN', 'NE', 'ID', 'WV', 'VT', 'WY', 'OK', 'SD')
AND EARNED_MONTH >= '202410' 
GROUP BY UW_AUTO.POL_ID, EARNED_MONTH 
ORDER BY UW_AUTO.POL_ID, EARNED_MONTH  
"""

# Buckets Dirty/Dirty and Dirty/Clean policies into the Dirty/Dirty bucket
cleanup_query = """
UPDATE USER_DB.AD1_PA08042.DATA_JULY
SET DERIVED_YES_NO_FLAG = 'Yes Acc/Viol/Comp'
WHERE CLEAN_DIRTY_CD = 'D'
"""

In [7]:
# Writes completed dashboard table to Snowflake based on the queries above
cs = ctx.cursor()
try:
    cs.execute(july_query)
    cs.execute(october_query)
    cs.execute(cleanup_query)
finally:
    cs.close()
ctx.close()
print('Done')

Done


In [8]:
full_data['RISK_ST_ABBR'].value_counts()

RISK_ST_ABBR
AZ    12881
IN     5174
MO     4832
CO     2956
TN     2822
MS     1205
WI     1007
UT      933
KY      666
MD      531
IA      465
LA      411
MN      355
NE      309
ID      299
WV      256
VT      192
WY       70
OK       63
SD       28
Name: count, dtype: int64